In [14]:
import sys
sys.path.append('../')

from src.finance_ml.data_preparation.data_preparation import DataLoader
from src.finance_ml.trading_strategy.pivot_strategy import *

In [15]:
dataloader = DataLoader(time_index_col= 'DATE', 
                    keep_cols = ['VOLUME','OPEN', 'CLOSE', 'LOW', 'TRANSACTIONS','HIGHT'])

data = dataloader.load_dataset({'AAPL':'../data/indices/SPY_2020-04-07_2022-04-06.parquet'})
data

,AAPL_VOLUME,AAPL_OPEN,AAPL_CLOSE,AAPL_LOW,AAPL_TRANSACTIONS,AAPL_HIGHT
DATE,,,,,,
2020-04-07 15:04:00,612879.0,268.72,269.140,268.3000,5753.0,269.18
2020-04-07 15:05:00,586743.0,269.14,269.380,268.8200,5555.0,269.49
2020-04-07 15:06:00,382167.0,269.38,269.420,269.1400,3427.0,269.55
2020-04-07 15:07:00,394478.0,269.44,269.860,269.1800,3679.0,269.91
2020-04-07 15:08:00,513549.0,269.85,269.775,269.7500,4273.0,270.10
...,...,...,...,...,...,...
2022-04-06 23:55:00,2392.0,445.41,445.370,445.3700,25.0,445.41
2022-04-06 23:56:00,2147.0,445.35,445.300,445.3000,26.0,445.35
2022-04-06 23:57:00,3647.0,445.32,445.240,445.1800,32.0,445.32


In [16]:
resampled_data_5min = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'5min')
resampled_data_15min = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'15min')
resampled_data_30min = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'30min')
resampled_data_1hr = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'1h')
resampled_data_1d = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'1d')
resampled_data_1wk = resample_ohlc(data,['AAPL_OPEN','AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE'],'1W')

D:\FAU\Ml in finance_ex\github_repo\notebooks\..\src\finance_ml\trading_strategy\pivot_strategy.py:82: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  resampled_data.index = resampled_data.index + pd.DateOffset(weekday=0)


In [17]:
traditional_pivot_1d = traditional_pivot(resampled_data_1d,'AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE')
traditional_pivot_1wk = traditional_pivot(resampled_data_1wk,'AAPL_HIGHT','AAPL_LOW','AAPL_CLOSE')

In [18]:
traditional_pivot_1d

,DATE,Pivot,S1,S2,S3,R1,R2,R3
1,2020-04-08,267.500000,260.610000,256.000000,249.110000,272.110000,279.000000,283.610000
2,2020-04-09,271.340000,266.680000,258.370000,253.710000,279.650000,284.310000,292.620000
6,2020-04-13,276.866667,272.533333,266.066667,261.733333,283.333333,287.666667,294.133333
7,2020-04-14,275.563333,272.926667,268.773333,266.136667,279.716667,282.353333,286.506667
8,2020-04-15,282.250000,279.600000,275.800000,273.150000,286.050000,288.700000,292.500000
...,...,...,...,...,...,...,...,...
723,2022-03-31,459.080333,456.264667,453.649333,450.833667,461.695667,464.511333,467.126667
724,2022-04-01,454.780000,449.580000,445.960000,440.760000,458.400000,463.600000,467.220000
727,2022-04-04,452.333333,450.096667,446.903333,444.666667,455.526667,457.763333,460.956667
728,2022-04-05,455.156667,453.383333,450.416667,448.643333,458.123333,459.896667,462.863333


In [19]:
mereged_data = merge_ohlc_pivot(resampled_data_15min,traditional_pivot_1d)

In [20]:
traded_buy = generate_buy(mereged_data,'AAPL_CLOSE','AAPL_OPEN')

In [21]:
traded_sell = generate_sell(mereged_data,'AAPL_CLOSE','AAPL_OPEN')

In [22]:
pnl_buy = get_pnl(traded_buy,'AAPL_CLOSE')

In [23]:
pnl_sell = get_pnl(traded_sell,'AAPL_CLOSE')

In [24]:
traded_buy.to_excel('test_buy.xlsx', index=False)

In [25]:
traded_sell.to_excel('test_sell.xlsx', index=False)